<a href="https://colab.research.google.com/github/jiin124/Dacon/blob/main/%EC%87%BC%ED%95%91%EB%AA%B0_%EC%A7%80%EC%A0%90%EB%B3%84_%EB%A7%A4%EC%B6%9C%EC%95%A1_%EC%98%88%EC%B8%A1_voting(cat%2Cxgb%2Clgbm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 쇼핑몰 지점별 매출액 예측

현재 86등 제일 나은 점수인 코드


https://dacon.io/competitions/official/235942/data

## train data
- id : 샘플 아이디
- Store : 쇼핑몰 지점
- Date : 주 단위(Weekly) 날짜
- Temperature : 해당 쇼핑몰 주변 기온
- Fuel_Price : 해당 쇼핑몰 주변 연료 가격
- Promotion 1~5 : 해당 쇼핑몰의 비식별화된 프로모션 정보
- Unemployment : 해당 쇼핑몰 지역의 실업률
- IsHoliday : 해당 기간의 공휴일 포함 여부
- Weekly_Sales : 주간 매출액 (목표 예측값)


## test data

- id : 샘플 아이디
- Store : 쇼핑몰 지점
- Date : 주 단위(Weekly) 날짜
- Temperature : 해당 쇼핑몰 주변 기온
- Fuel_Price : 해당 쇼핑몰 주변 연료 가격
- Promotion 1~5 : 해당 쇼핑몰의 비식별화된 프로모션 정보
- Unemployment : 해당 쇼핑몰 지역의 실업률
- IsHoliday : 해당 기간의 공휴일 포함 여부

어떤 프로모션을 진행해야 판매량이 올라갈까요?💥

매장 주변의 날씨가 판매량에 영향을 미치지는 않을까요?🌞

혹은 지역 사회의 실업률도 영향을 미칠 수 있지 않을까요?



쇼핑몰 매장별 프로모션 정보, 주변 날씨, 실업률, 연료 가격 등의 정보를 바탕으로

쇼핑몰의 각 지점별 주간 매출액을 예측해주세요!

## 해볼 것
- 군집분석


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [ ]:
#한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager,rc

font_path="/content/drive/MyDrive/NanumSquareB.ttf"
font=font_manager.FontProperties(fname=font_path)

In [ ]:
plt.rc('font',family='NanumBarunGothic')

NameError: ignored

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

In [3]:
train=pd.read_csv('/content/train.csv')
test=pd.read_csv('/content/test.csv')
sample=pd.read_csv('/content/sample_submission.csv')

In [ ]:
train.head()

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6255 entries, 0 to 6254
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            6255 non-null   int64  
 1   Store         6255 non-null   int64  
 2   Date          6255 non-null   object 
 3   Temperature   6255 non-null   float64
 4   Fuel_Price    6255 non-null   float64
 5   Promotion1    2102 non-null   float64
 6   Promotion2    1592 non-null   float64
 7   Promotion3    1885 non-null   float64
 8   Promotion4    1819 non-null   float64
 9   Promotion5    2115 non-null   float64
 10  Unemployment  6255 non-null   float64
 11  IsHoliday     6255 non-null   bool   
 12  Weekly_Sales  6255 non-null   float64
dtypes: bool(1), float64(9), int64(2), object(1)
memory usage: 592.6+ KB


In [ ]:
train.isna().sum()

id                 0
Store              0
Date               0
Temperature        0
Fuel_Price         0
Promotion1      4153
Promotion2      4663
Promotion3      4370
Promotion4      4436
Promotion5      4140
Unemployment       0
IsHoliday          0
Weekly_Sales       0
dtype: int64

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            180 non-null    int64  
 1   Store         180 non-null    int64  
 2   Date          180 non-null    object 
 3   Temperature   180 non-null    float64
 4   Fuel_Price    180 non-null    float64
 5   Promotion1    178 non-null    float64
 6   Promotion2    45 non-null     float64
 7   Promotion3    161 non-null    float64
 8   Promotion4    146 non-null    float64
 9   Promotion5    180 non-null    float64
 10  Unemployment  180 non-null    float64
 11  IsHoliday     180 non-null    bool   
dtypes: bool(1), float64(8), int64(2), object(1)
memory usage: 15.8+ KB


In [ ]:
test.isna().sum()

id                0
Store             0
Date              0
Temperature       0
Fuel_Price        0
Promotion1        2
Promotion2      135
Promotion3       19
Promotion4       34
Promotion5        0
Unemployment      0
IsHoliday         0
dtype: int64

# 1. 데이터 전처리

- NULL값 없애기
- 필요없는 변수 없애기(id)
- date 분리
- isholiday에서 날짜 처리


## id 없애기

In [4]:
train.drop('id',axis=1,inplace=True)

In [5]:
train.head()

,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68


In [6]:
test.drop('id',axis=1,inplace=True)
test.head()

,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday
0,1,05/10/2012,68.55,3.617,8077.89,NaN,18.22,3617.43,3626.14,6.573,False
1,1,12/10/2012,62.99,3.601,2086.18,NaN,8.11,602.36,5926.45,6.573,False
2,1,19/10/2012,67.97,3.594,950.33,NaN,4.93,80.25,2312.85,6.573,False
3,1,26/10/2012,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,6.573,False
4,2,05/10/2012,70.27,3.617,6037.76,NaN,10.04,3027.37,3853.40,6.170,False


## NULL값 처리
 promotion의 NAN값들이 많아 다 처리해주겠다. 0으로 처리하겠다. 

In [7]:
train=train.fillna(0)
train.isnull().sum()

Store           0
Date            0
Temperature     0
Fuel_Price      0
Promotion1      0
Promotion2      0
Promotion3      0
Promotion4      0
Promotion5      0
Unemployment    0
IsHoliday       0
Weekly_Sales    0
dtype: int64

In [8]:
train.head()

,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,8.106,False,1643690.90
1,1,12/02/2010,38.51,2.548,0.0,0.0,0.0,0.0,0.0,8.106,True,1641957.44
2,1,19/02/2010,39.93,2.514,0.0,0.0,0.0,0.0,0.0,8.106,False,1611968.17
3,1,26/02/2010,46.63,2.561,0.0,0.0,0.0,0.0,0.0,8.106,False,1409727.59
4,1,05/03/2010,46.50,2.625,0.0,0.0,0.0,0.0,0.0,8.106,False,1554806.68


In [9]:
test=test.fillna(0)
test.isnull().sum()

Store           0
Date            0
Temperature     0
Fuel_Price      0
Promotion1      0
Promotion2      0
Promotion3      0
Promotion4      0
Promotion5      0
Unemployment    0
IsHoliday       0
dtype: int64

## isholiday에서 숫자로 처리 

휴일은 1로 휴일이 아닌날은 0으로 처리하겠다. 

In [10]:
train['IsHoliday'].value_counts()

False    5805
True      450
Name: IsHoliday, dtype: int64

True가 휴일, False가 휴일이 아닌 것을 알 수 있다. 

In [11]:
train.columns

Index(['Store', 'Date', 'Temperature', 'Fuel_Price', 'Promotion1',
       'Promotion2', 'Promotion3', 'Promotion4', 'Promotion5', 'Unemployment',
       'IsHoliday', 'Weekly_Sales'],
      dtype='object')

In [12]:
mapping={True:1,False:0}

train['IsHoliday']=train['IsHoliday'].astype('object')
test['IsHoliday']=test['IsHoliday'].astype('object')


In [13]:
train['IsHoliday']=train['IsHoliday'].map(mapping)
test['IsHoliday']=test['IsHoliday'].map(mapping)

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6255 entries, 0 to 6254
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6255 non-null   int64  
 1   Date          6255 non-null   object 
 2   Temperature   6255 non-null   float64
 3   Fuel_Price    6255 non-null   float64
 4   Promotion1    6255 non-null   float64
 5   Promotion2    6255 non-null   float64
 6   Promotion3    6255 non-null   float64
 7   Promotion4    6255 non-null   float64
 8   Promotion5    6255 non-null   float64
 9   Unemployment  6255 non-null   float64
 10  IsHoliday     6255 non-null   int64  
 11  Weekly_Sales  6255 non-null   float64
dtypes: float64(9), int64(2), object(1)
memory usage: 586.5+ KB


In [15]:
train.head()

,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,8.106,0,1643690.90
1,1,12/02/2010,38.51,2.548,0.0,0.0,0.0,0.0,0.0,8.106,1,1641957.44
2,1,19/02/2010,39.93,2.514,0.0,0.0,0.0,0.0,0.0,8.106,0,1611968.17
3,1,26/02/2010,46.63,2.561,0.0,0.0,0.0,0.0,0.0,8.106,0,1409727.59
4,1,05/03/2010,46.50,2.625,0.0,0.0,0.0,0.0,0.0,8.106,0,1554806.68


## date 분리

In [16]:
train['date']=pd.to_datetime(train['Date'])

In [17]:
train.head()

,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,date
0,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,8.106,0,1643690.90,2010-05-02
1,1,12/02/2010,38.51,2.548,0.0,0.0,0.0,0.0,0.0,8.106,1,1641957.44,2010-12-02
2,1,19/02/2010,39.93,2.514,0.0,0.0,0.0,0.0,0.0,8.106,0,1611968.17,2010-02-19
3,1,26/02/2010,46.63,2.561,0.0,0.0,0.0,0.0,0.0,8.106,0,1409727.59,2010-02-26
4,1,05/03/2010,46.50,2.625,0.0,0.0,0.0,0.0,0.0,8.106,0,1554806.68,2010-05-03


In [18]:
test['date']=pd.to_datetime(test['Date'])

In [19]:
#date 삭제
train.drop('Date',axis=1,inplace=True)
test.drop('Date',axis=1,inplace=True)

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6255 entries, 0 to 6254
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         6255 non-null   int64         
 1   Temperature   6255 non-null   float64       
 2   Fuel_Price    6255 non-null   float64       
 3   Promotion1    6255 non-null   float64       
 4   Promotion2    6255 non-null   float64       
 5   Promotion3    6255 non-null   float64       
 6   Promotion4    6255 non-null   float64       
 7   Promotion5    6255 non-null   float64       
 8   Unemployment  6255 non-null   float64       
 9   IsHoliday     6255 non-null   int64         
 10  Weekly_Sales  6255 non-null   float64       
 11  date          6255 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(9), int64(2)
memory usage: 586.5 KB


# 데이터 전처리 2차

- 날짜 분리하기 

In [21]:
train['year']=train['date'].dt.year

In [22]:
train['month']=train['date'].dt.month
train['day']=train['date'].dt.day
train['weekday']=train['date'].dt.weekday

In [23]:
train.head()

,Store,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,date,year,month,day,weekday
0,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,8.106,0,1643690.90,2010-05-02,2010,5,2,6
1,1,38.51,2.548,0.0,0.0,0.0,0.0,0.0,8.106,1,1641957.44,2010-12-02,2010,12,2,3
2,1,39.93,2.514,0.0,0.0,0.0,0.0,0.0,8.106,0,1611968.17,2010-02-19,2010,2,19,4
3,1,46.63,2.561,0.0,0.0,0.0,0.0,0.0,8.106,0,1409727.59,2010-02-26,2010,2,26,4
4,1,46.50,2.625,0.0,0.0,0.0,0.0,0.0,8.106,0,1554806.68,2010-05-03,2010,5,3,0


In [24]:
test['year']=test['date'].dt.year
test['month']=test['date'].dt.month
test['day']=test['date'].dt.day
test['weekday']=test['date'].dt.weekday

In [25]:
test.head()

,Store,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,date,year,month,day,weekday
0,1,68.55,3.617,8077.89,0.00,18.22,3617.43,3626.14,6.573,0,2012-05-10,2012,5,10,3
1,1,62.99,3.601,2086.18,0.00,8.11,602.36,5926.45,6.573,0,2012-12-10,2012,12,10,0
2,1,67.97,3.594,950.33,0.00,4.93,80.25,2312.85,6.573,0,2012-10-19,2012,10,19,4
3,1,69.16,3.506,2585.85,31.75,6.00,1057.16,1305.01,6.573,0,2012-10-26,2012,10,26,4
4,2,70.27,3.617,6037.76,0.00,10.04,3027.37,3853.40,6.170,0,2012-05-10,2012,5,10,3


In [26]:
train.drop('date',axis=1,inplace=True)
test.drop('date',axis=1,inplace=True)

# 파생변수

- 뭐가 있을까

# 데이터 나누기

In [27]:
X_train=train.drop('Weekly_Sales',axis=1)
y_train=train['Weekly_Sales']
X_test=test

In [28]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)


(6255, 14)
(6255,)
(180, 14)


# 모델링



- 회귀
- 군집분석
- automl


In [29]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


## voting

161225.9425983884	220719/ 86위

In [42]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

estimators=[]

xgb=XGBRegressor()
estimators.append(xgb)

cat=CatBoostRegressor()
estimators.append(cat)

lgbm=LGBMRegressor()
estimators.append(lgbm)

params=[]

params_xgb={
    'n_estimators':[100,200],
    'learning_rate':[0.05],
    'min_child_weight':[7,10],
    'max_depth':[5,7],
    'subsample':[0.1],
    'colsample_bytree':[0.55]
}
params.append(params_xgb)

params_lgbm={
    'n_estimators':[100,150,200],
    #'max_depth':[3,5,7,9],
    #'min_data_in_leaf':[20,30,40,50],
    #'learning_rate':[0.1,0.05,0.001],

}
params.append(params_lgbm)

params_cat={

}
params.append(params_cat)

In [43]:
params

[{'colsample_bytree': [0.55],
  'learning_rate': [0.05],
  'max_depth': [5, 7],
  'min_child_weight': [7, 10],
  'n_estimators': [100, 200],
  'subsample': [0.1]},
 {'learning_rate': [0.1, 0.05, 0.001],
  'max_depth': [3, 5, 7, 9],
  'min_data_in_leaf': [20, 30, 40, 50],
  'n_estimators': [100, 150, 200]},
 {}]

In [44]:
estimators

[XGBRegressor(),
 LGBMRegressor()]

In [45]:
from tqdm.auto import tqdm#진행도를 확인하는 것

def gridSearchCV(models,params):
  best_models=[]

  for i in tqdm(range(0,len(models))):
    model_grid=GridSearchCV(models[i],params[i],n_jobs=-1,verbose=1,cv=3)
    model_grid.fit(X_train,y_train)
    best_models.append(model_grid.best_estimator_)
  return best_models


best_model_list=gridSearchCV(estimators,params)

  0%|          | 0/3 [00:00<?, ?it/s]

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[10:27:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Fitting 3 folds for each of 144 candidates, totalling 432 fits
0:	learn: 565122.7682847	total: 16.8ms	remaining: 3.33s
1:	learn: 564889.7777666	total: 29ms	remaining: 2.87s
2:	learn: 564662.2761578	total: 41.1ms	remaining: 2.7s
3:	learn: 564434.8000831	total: 52.8ms	remaining: 2.58s
4:	learn: 564193.2385003	total: 65.5ms	remaining: 2.55s
5:	learn: 563955.2754048	total: 76.9ms	remaining: 2.49s
6:	learn: 563687.9260111	total: 87.7ms	remaining: 2.42s
7:	learn: 563473.5515500	total: 109ms	remaining: 2.62s
8:	learn: 563207.5065596	total: 120ms	remaining: 2.56s
9:	learn: 562986.6560350	total: 132ms	remaining: 2.52s
10:	learn: 562765.8510286	total: 145ms	remaining: 2.48s
11:	learn: 562544.5786711	total: 156ms	remaining: 2.45s
12:	learn: 562316.8481411	total: 169ms	remaining: 2.42s
13:	learn: 562029.1767119	to

In [46]:
best_model_list

[XGBRegressor(colsample_bytree=0.55, learning_rate=0.05, max_depth=5,
              min_child_weight=10, subsample=0.1),
 LGBMRegressor()]

In [48]:
best_models=[
    ('xgb',XGBRegressor(colsample_bytree=0.55, learning_rate=0.05, max_depth=5,
              min_child_weight=10, subsample=0.1)),
    ('cat',CatBoostRegressor()),
    ('lgbm',LGBMRegressor(n_estimators=150))   
]

from sklearn.ensemble import VotingRegressor

voting_rg=VotingRegressor(estimators=best_models)
voting_rg.fit(X_train,y_train)

[10:34:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Learning rate set to 0.0547
0:	learn: 553043.1548784	total: 12.9ms	remaining: 12.9s
1:	learn: 544251.3688565	total: 26.2ms	remaining: 13.1s
2:	learn: 536521.6005065	total: 38.9ms	remaining: 12.9s
3:	learn: 530091.6826212	total: 47.2ms	remaining: 11.8s
4:	learn: 520712.1099056	total: 59.5ms	remaining: 11.8s
5:	learn: 513318.0130504	total: 70.4ms	remaining: 11.7s
6:	learn: 504790.6295304	total: 79.8ms	remaining: 11.3s
7:	learn: 499145.9158703	total: 89.5ms	remaining: 11.1s
8:	learn: 493199.6738460	total: 98.5ms	remaining: 10.8s
9:	learn: 488673.2019954	total: 111ms	remaining: 11s
10:	learn: 481479.0197497	total: 115ms	remaining: 10.3s
11:	learn: 474777.0225049	total: 118ms	remaining: 9.71s
12:	learn: 470820.1109345	total: 123ms	remaining: 9.33s
13:	learn: 466699.9010675	total: 126ms	remaining: 8.88s
14:	learn: 462721.2126843	total: 131ms	remaining: 8.59s
15:	lear

VotingRegressor(estimators=[('xgb',
                             XGBRegressor(colsample_bytree=0.55,
                                          learning_rate=0.05, max_depth=5,
                                          min_child_weight=10, subsample=0.1)),
                            ('cat',
                             <catboost.core.CatBoostRegressor object at 0x7f27648dfcd0>),
                            ('lgbm', LGBMRegressor())])

In [49]:
pred=voting_rg.predict(X_test)

In [50]:
sample4=sample.copy()
sample4['Weekly_Sales']=pred
sample4.to_csv('voting.csv',index=False)

## 모델링 XGBoostRegressor

현재  187138.95977 | 15위

In [ ]:
from tqdm.auto import tqdm
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

params={
    'n_estimators':[50,100,150,200,300,400],
    'learning_rate':[0.05,0.1,0.15,0.2],
    'min_child_weight':[1,5,7,10],
    'max_depth':[5,6,7,9],
    'subsample':[0.1,0.3],
    'colsample_bytree':[0.5,0.55,1]  
}

xgb=XGBRegressor(random_state=0,n_jobs=-1)
grid=GridSearchCV(xgb,param_grid=params,cv=3,n_jobs=-1)
grid.fit(X_train,y_train)

print('최적의 하이퍼 파라미터:',grid.best_params_)
print('최고의 예측 정확도:',grid.best_score_)


[02:57:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
최적의 하이퍼 파라미터: {'colsample_bytree': 0.5, 'learning_rate': 0.05, 'max_depth': 5, 'min_child_weight': 10, 'n_estimators': 100, 'subsample': 0.1}
최고의 예측 정확도: -0.35924845343060796


In [ ]:
pred=grid.predict(X_test)


In [ ]:
sample2=sample.copy()
sample2['Weekly_Sales']=pred
sample2.to_csv('xgb.csv',index=False)

In [ ]:
sample2

,id,Weekly_Sales
0,1,1.513573e+06
1,2,1.328201e+06
2,3,1.051834e+06
3,4,1.125155e+06
4,5,1.649692e+06
...,...,...
175,176,4.212480e+05
176,177,7.032872e+05
177,178,7.655172e+05
178,179,5.711506e+05


## 모델링 RandomForest

19위 | 	296844.0452271492	

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rf=RandomForestRegressor()
rf.fit(X_train,y_train)
pred=rf.predict(X_test)




In [ ]:
sample=pd.read_csv('/content/sample_submission.csv')

In [ ]:
sample1=sample.copy()
sample1['Weekly_Sales']=pred
sample1.to_csv('rf.csv',index=False)

In [ ]:
sample1

,id,Weekly_Sales
0,1,1.694335e+06
1,2,1.703553e+06
2,3,1.438050e+06
3,4,1.377913e+06
4,5,1.859579e+06
...,...,...
175,176,3.316599e+05
176,177,7.713822e+05
177,178,7.584580e+05
178,179,7.019198e+05


## 모델링 Catboost

In [ ]:
from 